# Dataset Scratch

In [1]:
## imports
import torch
from torch.utils.data import DataLoader  # Minibatches
import torchvision.datasets as datasets  # MNIST dataset
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn   # Nueral network modules.
from collections import OrderedDict

In [2]:
class NN(nn.Module):

    def __init__(self, input_size, middle_width, num_classes):


        super(NN, self).__init__()
        self.features = nn.Sequential(OrderedDict([
            ('hidden_layer', nn.Linear(input_size, middle_width)),
            ('hidden_activation', nn.ReLU()),
            ('readout', nn.Linear(middle_width, num_classes))
        ]))

    def forward(self, x):
        x = self.features(x)

        return x

Setting Device

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mnist Dataset

In [4]:
dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)

In [5]:
dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: dataset/
    Split: Train
    StandardTransform
Transform: ToTensor()

MNIST Dataset Function

In [6]:
def mnist_dataset(batch_size, train=True, values=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]):
    dataset = datasets.MNIST(root='dataset/', train=train, transform=transforms.ToTensor(), download=True)
    list = []
    targets_list = dataset.targets.tolist()
    for i in range(len(dataset)):
        if targets_list[i] in values:
            list.append(i)
    dataset1 = torch.utils.data.Subset(dataset, list)
    loader = DataLoader(dataset=dataset1, batch_size=batch_size, shuffle=True)
    return loader

MNIST Subset Class

In [7]:
## extend from torch dataset.

class CustomDataset:
    def __init__(self, data, target):
        self.data = data
        self.targets = target
        
    def __len__(self):
        return self.data
    
    def __getitem__(self, idx):
        current_sample = self.data[idx, :]
        current_target = self.targets[idx]
        return {
            "sample":torch.tensor(current_sample, dtype=torch.float),
            "target": torch.tensor(current_target, dtype=torch.long)
        }

Function that changes the clasification targets of MNSIT values

In [8]:
def classify(targets, values):
    for key, element in enumerate(values):
        targets[targets == element] = key
    return targets

TEST

In [9]:
model = NN(input_size=784,
           middle_width=100,
            num_classes=2).to(device=device)

In [10]:
# Loading dataset
loader = mnist_dataset(100, values=[2, 7])

In [11]:
loss_function = nn.CrossEntropyLoss()

In [12]:
# Getting datapoints for training (but not training)
for batch_idx, (data, targets) in enumerate(loader):
    data = data.reshape(data.shape[0], -1).to(device=device)
    targets = targets.to(device=device)
    scores = model(data)
    loss = loss_function(scores, classify(targets, [2, 7]))